# Boolean Calculator: Top-down Parser


## Translation scheme

|Translation scheme                                                                                   |
|-----------------------------------------------------------------------------------------------------|
| entry -> `print` exprOR `;`{write('The result is {exprOR.s} ;')}                                    |
| entry -> asign `;`                                                                                    |
| asign -> `ID` `=` exprOR   { Table\[ID\] = exprOR.s }                                                 |
| exprOR -> exprAND { exprOR'.h = exprAND.s} exprOR' { exprOR.s = eprOR' }                            |
| exprOR' -> `or` exprAND { exprOR_1'.h = exprOR.h or exprAND.s } exprOR_1' { exprOR'.s = exprOR_1.s }|
| exprOR' -> e { exprOR'.s = exprOR'.h }                                                              |
| exprAND -> boolean { exprAND'.h = boolean.s } exprAND' { exprAND.s = exprAND'.s }                   |
| exprAND' -> `and` boolean { exprAND_1.h = exprAND'.h and boolean.s} exprAND_1'{exprAND'.s = exprAND_1'.s } |
| exprAND' -> e { exprAND'.s = exprAND'.h }                                                           |
| boolean -> not boolean_1 { boolean.s = not boolean_1.s }                                            |
| boolean -> `CBOOLEAN` { boolean.s = CBOOLEAN.lexval }                                               |
| boolean -> `ID` { boolean.s = table\[ID.lexval\] }                                                    |
| boolean -> `(` exprOR `)` { boolean.s = exprOR.s }                                                  |


## First and Follow sets

| Sintactic rules                    |
|------------------------------------|
| entry -> `print` exprOR `;`        |
| entry -> asign `;`                   |
| asign -> `ID` `=` exprOR           |
| exprOR -> exprAND exprOR'          |
| exprOR' -> `or` exprAND exprOR'    |
| exprOR' -> e                       |
| exprAND -> boolean exprAND'        |
| exprAND' -> `and` boolean exprAND' |
| exprAND' -> e                      |
| boolean -> not boolean             |
| boolean -> `CBOOLEAN`              |
| boolean -> `ID`                    |
| boolean -> `(` exprOR `)`          |


### First set

```
First(entry) = { Print , ID}
First(asign) = { ID }
First(ExprOR) = { not, CBOOLEAN, ID, '(' }
First(ExprOR') = { 'or', e }
First(ExprAND) = { not, CBOOLEAN, ID, '(' }
First(ExprAND') = { and, e }
First(Boolean) = {not, CBOOLEAN, ID, '(' }
```

### Follow set
```
Follow(entry) = {$}
Follow(asign) = {';'}
Follow(exprOR) = Follow(asign) U {')'} = { ';', ')' }
Follow(exprOR') = Follow(exprOR) U Follow(exprOR')  = { ';', ')' }
Follow(exprAND) = First(exprOR') - {e} U Follow(exprOR') = {or, ';', ')' }
Follow(exprAND') = Follow(exprAND) U Follow(exprAND') = {or, ';', ')' }
Follow(boolean) = First(exprAND') - {e} U  Follow(exprAND) U Follow(boolean) U { ')' } = {and, or, ';', ')'}
```

# Necessary libraries

In [1]:
!pip install sly

  Using cached sly-0.4.tar.gz (60 kB)
Using legacy 'setup.py install' for sly, since package 'wheel' is not installed.
    Running setup.py install for sly: started
    Running setup.py install for sly: finished with status 'done'
You should consider upgrading via the 'c:\users\los_g\onedrive\documentos\pl\pl-2020\prácticas\boolean-calculator-sintax-direct-translation\.env\scripts\python.exe -m pip install --upgrade pip' command.


# Lexer implementation

In [12]:
from sly import Lexer

class BooleanCalcLexer(Lexer):
    tokens = {ID, CBOOLEAN, ASG, NOT, OR, AND}
    ignore = ' \t'
    literals = {';', '(', ')'}

    # Regular expression rules for tokens
    ID = r'[a-zA-Z_][a-zA-Z0-9]*'
    ID['not'] = NOT
    ID['and'] = AND
    ID['or'] = OR
    ID['false'] = CBOOLEAN
    ID['true'] = CBOOLEAN
    ASG = r':='

    @_(r'\n+')
    def newline(self, t):
        self.lineno += t.value.count('\n')
    
    def error(self, t):
        print("line {}: Bad character %r" % (self.lineno, t.value[0]))
        self.index += 1

In [13]:
    if __name__ == '__main__':
        data = 'z := true or not (x and not y);'
        lexer = BooleanCalcLexer()
        for token in lexer.tokenize(data):
            print(token)

Token(type='ID', value='z', lineno=1, index=0)
Token(type='ASG', value=':=', lineno=1, index=2)
Token(type='CBOOLEAN', value='true', lineno=1, index=5)
Token(type='OR', value='or', lineno=1, index=10)
Token(type='NOT', value='not', lineno=1, index=13)
Token(type='(', value='(', lineno=1, index=17)
Token(type='ID', value='x', lineno=1, index=18)
Token(type='AND', value='and', lineno=1, index=20)
Token(type='NOT', value='not', lineno=1, index=24)
Token(type='ID', value='y', lineno=1, index=28)
Token(type=')', value=')', lineno=1, index=29)
Token(type=';', value=';', lineno=1, index=30)
